In [1]:
import moviepy.editor as mp
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.video.io.VideoFileClip import VideoFileClip
import speech_recognition as sr
from tkinter.filedialog import askopenfilename
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
import json

In [24]:
name_file = "Borja"
n_file = 2

In [26]:
# Extraigo el audio del vídeo

video = mp.VideoFileClip(".\\videos\\{}_{}.mp4".format(name_file, n_file))
audio = video.audio.write_audiofile(r".\\audios\\{}_{}.wav".format(name_file, n_file)) # de cuando se recibia un vídeo
recibido_cambio = sr.Recognizer()
speech_audio = sr.AudioFile(".\\audios\\{}_{}.wav".format(name_file, n_file))

chunk:   0%|          | 0/891 [00:00<?, ?it/s, now=None]

MoviePy - Writing audio in .\\audios\\Borja_2.wav


MoviePy - Done.


In [27]:
# Reconocedor básico

with speech_audio as fuente:
    #Se reduce el ruido
    recibido_cambio.adjust_for_ambient_noise(fuente)
    audio = recibido_cambio.record(fuente)
    #Transcripcion
    texto_transcrito = recibido_cambio.recognize_google(audio, language = "es-ES")
    print(texto_transcrito)

acelera un aproximado funda mantengo el acelerador rotonda media celador centro ocupado


In [28]:
# Guardo el texto en bruto transcrito de los audios

filename_t_raw = ".\\t_raw\\{}_{}.txt".format(name_file, n_file)
with open(filename_t_raw, "w", encoding="utf-8") as f:
    f.write(texto_transcrito)
print("Se ha guardado la transcripción en el archivo " + filename_t_raw)

Se ha guardado la transcripción en el archivo .\t_raw\Borja_2.txt


In [29]:
f = open ('.\\t_processed\\{}_{}.txt'.format(name_file, n_file),'r',encoding='utf8')
text = f.read()

sw = set(stopwords.words("spanish"))
# Limpio con regex el texto
text = re.sub('[%s]' % re.escape(string.punctuation + "'" + '"' + "’" + '”' + '“' + "•‘"), ' ', str(text))
text = re.sub('\w*\d\w*', ' ', str(text))
# Pongo el texto en minúsculas
text = text.lower()
# Tokenizo por palabras
text = word_tokenize(str(text), language='spanish')
tokens = []
for t in text:
    if not t in sw:
        tokens.append(t)

json_filename = ".\\t_processed\\{}_{}.json".format(name_file, n_file)
with open(json_filename, "w", encoding='utf-8') as f:
    data = json.dump([], f)
with open(json_filename, "r", encoding='utf-8') as f:
    data = json.load(f)
for t in tokens:
    entry = {
        'word': t,
        'token': SnowballStemmer('spanish').stem(t),
        'start': 0.0
    }
    data.append(entry)

with open(json_filename, "w", encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False)
print(data)

[{'word': 'acelero', 'token': 'aceler', 'start': 0.0}, {'word': 'aproximo', 'token': 'aproxim', 'start': 0.0}, {'word': 'rotonda', 'token': 'rotond', 'start': 0.0}, {'word': 'mantengo', 'token': 'manteng', 'start': 0.0}, {'word': 'acelerador', 'token': 'aceler', 'start': 0.0}, {'word': 'rotonda', 'token': 'rotond', 'start': 0.0}, {'word': 'media', 'token': 'medi', 'start': 0.0}, {'word': 'mantengo', 'token': 'manteng', 'start': 0.0}, {'word': 'acelerador', 'token': 'aceler', 'start': 0.0}, {'word': 'rotonda', 'token': 'rotond', 'start': 0.0}, {'word': 'cerca', 'token': 'cerc', 'start': 0.0}, {'word': 'centro', 'token': 'centr', 'start': 0.0}, {'word': 'ocupado', 'token': 'ocup', 'start': 0.0}, {'word': 'levanto', 'token': 'levant', 'start': 0.0}, {'word': 'acelerador', 'token': 'aceler', 'start': 0.0}, {'word': 'izquierda', 'token': 'izquierd', 'start': 0.0}, {'word': 'libre', 'token': 'libr', 'start': 0.0}, {'word': 'entro', 'token': 'entro', 'start': 0.0}, {'word': 'rotonda', 'token'

In [30]:
instrucciones = {
    'aceler': 'T-ON',
    'acerc rotond': 'APROX',
    'rotond medi': 'RND-MD',
    'manteng aceler': 'T-HOLD',
    'rotond cerc': 'RND-NR',
    'izquierd libr': 'L-FREE',
    'izquierd ocup': 'L-BUSY',
    'ced': 'RND-IN',
    'entro rotond': 'RND-IN',
    'gir derech': 'TURN-R',
    'intermitent izquierd': 'LB-ON',
    'rotond gir izquierd': 'TURN-L',
    'rotond gir derech': 'TURN-R',
    'gir izquierd': 'TURN-L',
    'quit intermitent': 'BLK-OFF',
    'intermitent derech': 'RB-ON',
    'salg rotond': 'RND-EXIT',
    'aproxim rotond': 'APROX',
    'levant aceler': 'T-OFF',
    'levant pie aceler': 'T-OFF',
    'vien nadi': 'RND-CLEAR',
    'gir izquierd rotond': 'TURN-L',
    'gir derech rotond': 'TURN-R',
    'atencion ciclist': 'WARN-VEH',
    'atencion coche': 'WARN-VEH',
    'fren': 'B-ON',
    'centr libr': 'L-FREE',
    'centr ocup': 'F-BUSY',
    'enderez': 'TURN-STR'
}

json_filename = ".\\t_processed\\{}_{}.json".format(name_file, n_file)
with open(json_filename, "r", encoding='utf-8') as f:
    data = json.load(f)

dataset = {}
count = 0

for i, x in enumerate(data):
    w = x['token']
    if x['token'] in instrucciones:
        #print("{} => {}".format(w, instrucciones[w]))
        dataset["inst_{}".format(count)] = [x]
        dataset["inst_{}".format(count)].append({"tag": instrucciones[w], "sentence": x['word']})
        count += 1
    elif (len(data) > i + 2) and x['token'] + ' ' + data[i + 1]['token'] + ' ' + data[i + 2]['token'] in instrucciones:
        sentence = x['word'] + ' ' + data[i + 1]['word'] + ' ' + data[i + 2]['word']
        sentence_t = x['token'] + ' ' + data[i + 1]['token'] + ' ' + data[i + 2]['token']
        dataset["inst_{}".format(count)] = [x, data[i + 1], data[i + 2]]
        dataset["inst_{}".format(count)].append({"tag": instrucciones[sentence_t], "sentence": sentence})
        count += 1
        del data[i + 1]
        del data[i + 1]
        #print("{} => {}".format(sentence, instrucciones[sentence]))
    elif (len(data) > i + 1) and x['token'] + ' ' + data[i + 1]['token'] in instrucciones:
        sentence = x['word'] + ' ' + data[i + 1]['word']
        sentence_t = x['token'] + ' ' + data[i + 1]['token']
        dataset["inst_{}".format(count)] = [x, data[i + 1]]
        dataset["inst_{}".format(count)].append({"tag": instrucciones[sentence_t], "sentence": sentence})
        count += 1
        del data[i + 1]
        #print("{} => {}".format(sentence, instrucciones[sentence]))

json_out = ".\\t_tags\\{}_{}.json".format(name_file, n_file)
with open(json_out, "w", encoding='utf-8') as f:
    json.dump(dataset, f, ensure_ascii=False)

print(dataset)

{'inst_0': [{'word': 'acelero', 'token': 'aceler', 'start': 0.0}, {'tag': 'T-ON', 'sentence': 'acelero'}], 'inst_1': [{'word': 'aproximo', 'token': 'aproxim', 'start': 0.0}, {'word': 'rotonda', 'token': 'rotond', 'start': 0.0}, {'tag': 'APROX', 'sentence': 'aproximo rotonda'}], 'inst_2': [{'word': 'mantengo', 'token': 'manteng', 'start': 0.0}, {'word': 'acelerador', 'token': 'aceler', 'start': 0.0}, {'tag': 'T-HOLD', 'sentence': 'mantengo acelerador'}], 'inst_3': [{'word': 'rotonda', 'token': 'rotond', 'start': 0.0}, {'word': 'media', 'token': 'medi', 'start': 0.0}, {'tag': 'RND-MD', 'sentence': 'rotonda media'}], 'inst_4': [{'word': 'mantengo', 'token': 'manteng', 'start': 0.0}, {'word': 'acelerador', 'token': 'aceler', 'start': 0.0}, {'tag': 'T-HOLD', 'sentence': 'mantengo acelerador'}], 'inst_5': [{'word': 'rotonda', 'token': 'rotond', 'start': 0.0}, {'word': 'cerca', 'token': 'cerc', 'start': 0.0}, {'tag': 'RND-NR', 'sentence': 'rotonda cerca'}], 'inst_6': [{'word': 'centro', 'toke